In [2]:
with open('..//data//data.json') as file:
        data = json.load(file)

    # Extract relevant fields and store them in a list
items = []
for item in data:
        print(item)

{'id': 28, 'product_name': 'Rose Bouquet', 'price': 450, 'image': '/media/product_images/w1_xj73zIb.webp', 'product_desc': 'The red rose symbolizes love, beauty, and courage, linked to Venus, the planet of love. White roses signify purity and new beginnings, pink represents grace and admiration, and orange expresses passion and enthusiasm.', 'types': 'Wedding', 'categories': 'Rose', 'color_available': ['red', 'yellow', 'green', 'gray'], 'size': ['M', 'S', 'L']}
{'id': 29, 'product_name': 'Lavender Bouquet', 'price': 500, 'image': '/media/product_images/w2_ayBnSJF.webp', 'product_desc': "Lavender symbolizes love, devotion, and fertility, promoting calm and peace. Traditionally included in Irish wedding bouquets, it's believed to bring happiness and luck to a marriage.", 'types': 'Wedding', 'categories': 'Lavender', 'color_available': ['red', 'yellow', 'green', 'gray'], 'size': ['M', 'S', 'L']}
{'id': 30, 'product_name': 'Orchid Bouquet', 'price': 600, 'image': '/media/product_images/w5_

In [10]:
import json
from langchain_core.documents import Document  # Ensure this import is correct

def dataconverter():
    # Load the JSON data from the file
    with open('..//data//data.json') as file:
        data = json.load(file)

    # Extract relevant fields and store them in a list
    items = []
    for item in data:
        item_info = {
            'product_name': item.get('product_name'),
            'product_desc': item.get('product_desc'),
            'price': item.get('price'),
            'types': item.get('types'),
            'categories': item.get('categories'),
            'color_available': item.get('color_available'),
            'size':item.get('size')
        }
        items.append(item_info)
    
    # Create documents
    docs = []
    for entry in items:
        metadata = {"product_name": entry['product_name']}
        
        # Convert the fields to a string format (JSON format)
        page_content = json.dumps({
            'product_desc': entry['product_desc'],
            'price': entry['price'],
            'types': entry['types'],
            'categories': entry['categories'],
            'color_available': entry['color_available'],
            'size':entry['size']
        })
        
        # Create the Document object with page content and metadata
        doc = Document(page_content=page_content, metadata=metadata)
        
        # Append the doc to the list
        docs.append(doc)
    print(docs)
    return docs  # Return the list of documents




In [11]:
dataconverter()

[Document(metadata={'product_name': 'Rose Bouquet'}, page_content='{"product_desc": "The red rose symbolizes love, beauty, and courage, linked to Venus, the planet of love. White roses signify purity and new beginnings, pink represents grace and admiration, and orange expresses passion and enthusiasm.", "price": 450, "types": "Wedding", "categories": "Rose", "color_available": ["red", "yellow", "green", "gray"], "size": ["M", "S", "L"]}'), Document(metadata={'product_name': 'Lavender Bouquet'}, page_content='{"product_desc": "Lavender symbolizes love, devotion, and fertility, promoting calm and peace. Traditionally included in Irish wedding bouquets, it\'s believed to bring happiness and luck to a marriage.", "price": 500, "types": "Wedding", "categories": "Lavender", "color_available": ["red", "yellow", "green", "gray"], "size": ["M", "S", "L"]}'), Document(metadata={'product_name': 'Orchid Bouquet'}, page_content='{"product_desc": "Orchids symbolize love, beauty, strength, and luxury

[Document(metadata={'product_name': 'Rose Bouquet'}, page_content='{"product_desc": "The red rose symbolizes love, beauty, and courage, linked to Venus, the planet of love. White roses signify purity and new beginnings, pink represents grace and admiration, and orange expresses passion and enthusiasm.", "price": 450, "types": "Wedding", "categories": "Rose", "color_available": ["red", "yellow", "green", "gray"], "size": ["M", "S", "L"]}'),
 Document(metadata={'product_name': 'Lavender Bouquet'}, page_content='{"product_desc": "Lavender symbolizes love, devotion, and fertility, promoting calm and peace. Traditionally included in Irish wedding bouquets, it\'s believed to bring happiness and luck to a marriage.", "price": 500, "types": "Wedding", "categories": "Lavender", "color_available": ["red", "yellow", "green", "gray"], "size": ["M", "S", "L"]}'),
 Document(metadata={'product_name': 'Orchid Bouquet'}, page_content='{"product_desc": "Orchids symbolize love, beauty, strength, and luxu

In [12]:
from langchain_astradb import AstraDBVectorStore
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings
#from happybot.dataconvertert import dataconverter
import os


load_dotenv()

GENAI_API_KEY=os.getenv("API_KEY")
ASTRA_DB_API_ENDPOINT=os.getenv("ASTRA_DB_API_END_POINT")
ASTRA_DB_APPLICATION_TOKEN=os.getenv("ASTRA_DB_APPLICATION_TOKENS")
ASTRA_DB_KEYSPACE=os.getenv("ASTRA_DB_KEYSPACE")

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GENAI_API_KEY)

def ingestdata(status):
    vstore = AstraDBVectorStore(
            embedding=embedding,
            collection_name="chatbot_db",
            api_endpoint=ASTRA_DB_API_ENDPOINT,
            token=ASTRA_DB_APPLICATION_TOKEN,
            namespace=ASTRA_DB_KEYSPACE,
        )
    
    storage=status
    
    if storage==None:
        docs=dataconverter()
        inserted_ids = vstore.add_documents(docs)
    else:
        return vstore
    return vstore, inserted_ids

if __name__=='__main__':
    vstore,inserted_ids=ingestdata(None)
    print(f"\nInserted {len(inserted_ids)} documents.")
    results = vstore.similarity_search("can you tell me the low budget sound basshead.")
    for res in results:
            print(f"* {res.page_content} [{res.metadata}]")
            

d:\Sunway\Fyp\FlowerSellingE-commerce\chatbot\chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'product_name': 'Rose Bouquet'}, page_content='{"product_desc": "The red rose symbolizes love, beauty, and courage, linked to Venus, the planet of love. White roses signify purity and new beginnings, pink represents grace and admiration, and orange expresses passion and enthusiasm.", "price": 450, "types": "Wedding", "categories": "Rose", "color_available": ["red", "yellow", "green", "gray"], "size": ["M", "S", "L"]}'), Document(metadata={'product_name': 'Lavender Bouquet'}, page_content='{"product_desc": "Lavender symbolizes love, devotion, and fertility, promoting calm and peace. Traditionally included in Irish wedding bouquets, it\'s believed to bring happiness and luck to a marriage.", "price": 500, "types": "Wedding", "categories": "Lavender", "color_available": ["red", "yellow", "green", "gray"], "size": ["M", "S", "L"]}'), Document(metadata={'product_name': 'Orchid Bouquet'}, page_content='{"product_desc": "Orchids symbolize love, beauty, strength, and luxury

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
#from happybot.data_ingest import ingestdata
import os
from dotenv import load_dotenv

load_dotenv()

GENAI_API_KEY=os.getenv("API_KEY")

def generation(vstore):
    retriever = vstore.as_retriever(search_kwargs={"k": 3})

    FLOWER_BOT_TEMPLATE = """
    Your ecommerce bot is an expert in product recommendations and customer queries.
    It analyzes product name, price,types, categories, description, color_available and size to provide accurate and helpful responses.
    Ensure your answers are relevant to the product context and refrain from straying off-topic.
    Your responses should be concise and informative.

    CONTEXT:
    {context}

    QUESTION: {question}

    YOUR ANSWER:
    """


    prompt = ChatPromptTemplate.from_template(FLOWER_BOT_TEMPLATE)

    # Specify the model for ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GENAI_API_KEY,temperature=0.3)

    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    return chain

if __name__ == '__main__':
    vstore = ingestdata("done")
    chain = generation(vstore)
    print(chain.invoke("can you provide a provide types of flowers"))


- flowers
- Wedding
